In [1]:
import pandas as pd

# 엑셀 파일 경로 지정
file_path = '(0203최종_2)merged_final_result.xlsx'

# 엑셀 파일 읽기
xls = pd.ExcelFile(file_path)

# 각 시트의 상위 25% 행들을 저장할 리스트
filtered_data_frames = []

# 모든 시트에 대해 작업 수행
for sheet_name in xls.sheet_names:
    # 시트 데이터를 읽어옴
    df = pd.read_excel(xls, sheet_name=sheet_name)

    # 열 번호로 E열(5번째 열)부터 Z열(26번째 열)까지 선택
    columns_to_sum = df.iloc[:, 4:26]  # 0부터 시작하는 인덱스로 4번째부터 25번째 열까지

    # E열부터 Z열까지의 합산
    df['sum_E_to_Z'] = columns_to_sum.sum(axis=1)

    # 합계 기준 상위 25% 행들만 선택
    threshold = df['sum_E_to_Z'].quantile(0.75)
    filtered_df = df[df['sum_E_to_Z'] >= threshold]

    # 필요한 결과만 저장
    filtered_data_frames.append(filtered_df)

# A~D열 기준으로 공통 조합 찾기
common_keys = filtered_data_frames[0][filtered_data_frames[0].columns[:4]].copy()

for df in filtered_data_frames[1:]:
    common_keys = pd.merge(common_keys, df[df.columns[:4]], how='inner')

# 최종 결과 저장을 위한 딕셔너리
final_results = {}

# 각 시트에서 공통된 조합에 해당하는 모든 데이터를 추출
for i, sheet_name in enumerate(xls.sheet_names):
    # 현재 시트 데이터
    df = filtered_data_frames[i]

    # 공통 조합에 해당하는 행들만 가져옴
    result_df = pd.merge(common_keys, df, how='inner', on=df.columns[:4].tolist())

    # 결과 저장
    final_results[sheet_name] = result_df

# 하나의 엑셀 파일로 저장
output_file = '(0203최종_2)상위25%_교집합_result.xlsx'
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, result_df in final_results.items():
        result_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"결과가 파일 '{output_file}'에 저장되었습니다.")


결과가 파일 '(0203최종_2)상위25%_교집합_result.xlsx'에 저장되었습니다.
